# Setting up the SQL Database

It had to happen eventually I guess.  I'm not paying for compute at this stage.  Maybe I really should just buy an instance with a shit ton of RAM.  Even with this EDA is going to be weird as shit. At least I've decided what to do.

In [1]:
import pandas as pd
import sqlalchemy

In [2]:
train = pd.read_csv('./data/train.csv', chunksize=50000)

In [3]:
df = train.get_chunk()

In [4]:
df.head()

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385
1,Organic Search,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147
2,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,1472865386
3,Organic Search,20160902,"{""browser"": ""UC Browser"", ""browserVersion"": ""n...",4763447161404445595,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",4763447161404445595_1472881213,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472881213,1,1472881213
4,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",27294437909732085,"{""continent"": ""Europe"", ""subContinent"": ""North...",27294437909732085_1472822600,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472822600,2,1472822600


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
channelGrouping         50000 non-null object
date                    50000 non-null int64
device                  50000 non-null object
fullVisitorId           50000 non-null object
geoNetwork              50000 non-null object
sessionId               50000 non-null object
socialEngagementType    50000 non-null object
totals                  50000 non-null object
trafficSource           50000 non-null object
visitId                 50000 non-null int64
visitNumber             50000 non-null int64
visitStartTime          50000 non-null int64
dtypes: int64(4), object(8)
memory usage: 4.6+ MB


## Looking at JSON columns

I have no idea what all the fields are in these columns.  The good news is that missing values from these are assumed to be obvious; lack of a field means absence from the visit, if it's censored it will say so, these all should have been collected by automation.  Unfortunately, Kaggle decided against telling me the names of all possible fields, which I probably wanted before I decided on a schema.  So here is that part gdi. Maybe there was a more clever way to deal with these jsons than just enumerate all columns and fill those in ahead of time, but tht aint me

In [6]:
import re

In [7]:
[re.split('\"(w*)', string)[2] for string in df.iloc[5,:]['trafficSource'].split(',')]

['campaign', 'source', 'medium', 'keyword', 'adwordsClickInfo']

In [8]:
totals_fields = set()
device_fields = set()
geoNetwork_fields = set()
trafficSource_fields = set()

field_dict = {'device':device_fields, 
              'geoNetwork':geoNetwork_fields, 
              'totals':totals_fields, 
              'trafficSource':trafficSource_fields}

When correctly formatted, we find the word in the second match group.  Sometimes, we have empty dictionaries, which will throw an error.  The current expression and index will give us the field names by inspection, and we assume JSONs will have a consistent output.

In [9]:
while len(df)>0:
    for _, row in df.iterrows():
        for field_name, field_set in field_dict.items():
            try:
                cur_fields = [re.split('\"(w*)', string)[2] for string in row[field_name].split(',')]
                field_set.update(cur_fields)
            except:
                pass
    print(df.tail(1).index)
    try:
        df = train.get_chunk()
    except:
        break

RangeIndex(start=49999, stop=50000, step=1)
RangeIndex(start=99999, stop=100000, step=1)
RangeIndex(start=149999, stop=150000, step=1)
RangeIndex(start=199999, stop=200000, step=1)
RangeIndex(start=249999, stop=250000, step=1)
RangeIndex(start=299999, stop=300000, step=1)
RangeIndex(start=349999, stop=350000, step=1)
RangeIndex(start=399999, stop=400000, step=1)
RangeIndex(start=449999, stop=450000, step=1)
RangeIndex(start=499999, stop=500000, step=1)
RangeIndex(start=549999, stop=550000, step=1)
RangeIndex(start=599999, stop=600000, step=1)
RangeIndex(start=649999, stop=650000, step=1)
RangeIndex(start=699999, stop=700000, step=1)
RangeIndex(start=749999, stop=750000, step=1)
RangeIndex(start=799999, stop=800000, step=1)
RangeIndex(start=849999, stop=850000, step=1)
RangeIndex(start=899999, stop=900000, step=1)
RangeIndex(start=903652, stop=903653, step=1)


In [10]:
field_dict

{'device': {'browser',
  'browserSize',
  'browserVersion',
  'deviceCategory',
  'flashVersion',
  'isMobile',
  'language',
  'mobileDeviceBranding',
  'mobileDeviceInfo',
  'mobileDeviceMarketingName',
  'mobileDeviceModel',
  'mobileInputSelector',
  'operatingSystem',
  'operatingSystemVersion',
  'screenColors',
  'screenResolution'},
 'geoNetwork': {'',
  'city',
  'cityId',
  'continent',
  'country',
  'latitude',
  'longitude',
  'metro',
  'networkDomain',
  'networkLocation',
  'region',
  'subContinent'},
 'totals': {'bounces',
  'hits',
  'newVisits',
  'pageviews',
  'transactionRevenue',
  'visits'},
 'trafficSource': {'',
  'adContent',
  'adNetworkType',
  'adwordsClickInfo',
  'campaign',
  'campaignCode',
  'criteriaParameters',
  'gclId',
  'isTrueDirect',
  'isVideoAd',
  'keyword',
  'medium',
  'referralPath',
  'slot',
  'source',
  'targetingCriteria'}}

To be fair, we should also examine the test data.

In [13]:
test = pd.read_csv('./data/test.csv', chunksize=50000)

In [16]:
df = test.get_chunk()

In [14]:
totals_fields_test = set()
device_fields_test = set()
geoNetwork_fields_test = set()
trafficSource_fields_test = set()

field_dict_test = {'device':device_fields_test, 
              'geoNetwork':geoNetwork_fields_test, 
              'totals':totals_fields_test, 
              'trafficSource':trafficSource_fields_test}

In [17]:
while len(df)>0:
    for _, row in df.iterrows():
        for field_name, field_set in field_dict_test.items():
            try:
                cur_fields = [re.split('\"(w*)', string)[2] for string in row[field_name].split(',')]
                field_set.update(cur_fields)
            except:
                pass
    print(df.tail(1).index)
    try:
        df = test.get_chunk()
    except:
        break

RangeIndex(start=49999, stop=50000, step=1)
RangeIndex(start=99999, stop=100000, step=1)
RangeIndex(start=149999, stop=150000, step=1)
RangeIndex(start=199999, stop=200000, step=1)
RangeIndex(start=249999, stop=250000, step=1)
RangeIndex(start=299999, stop=300000, step=1)
RangeIndex(start=349999, stop=350000, step=1)
RangeIndex(start=399999, stop=400000, step=1)
RangeIndex(start=449999, stop=450000, step=1)
RangeIndex(start=499999, stop=500000, step=1)
RangeIndex(start=549999, stop=550000, step=1)
RangeIndex(start=599999, stop=600000, step=1)
RangeIndex(start=649999, stop=650000, step=1)
RangeIndex(start=699999, stop=700000, step=1)
RangeIndex(start=749999, stop=750000, step=1)
RangeIndex(start=799999, stop=800000, step=1)
RangeIndex(start=804683, stop=804684, step=1)


In [18]:
field_dict_test

{'device': {'browser',
  'browserSize',
  'browserVersion',
  'deviceCategory',
  'flashVersion',
  'isMobile',
  'language',
  'mobileDeviceBranding',
  'mobileDeviceInfo',
  'mobileDeviceMarketingName',
  'mobileDeviceModel',
  'mobileInputSelector',
  'operatingSystem',
  'operatingSystemVersion',
  'screenColors',
  'screenResolution'},
 'geoNetwork': {'',
  'city',
  'cityId',
  'continent',
  'country',
  'latitude',
  'longitude',
  'metro',
  'networkDomain',
  'networkLocation',
  'region',
  'subContinent'},
 'totals': {'bounces', 'hits', 'newVisits', 'pageviews', 'visits'},
 'trafficSource': {'',
  'adContent',
  'adNetworkType',
  'adwordsClickInfo',
  'campaign',
  'criteriaParameters',
  'gclId',
  'isTrueDirect',
  'isVideoAd',
  'keyword',
  'medium',
  'referralPath',
  'slot',
  'source',
  'targetingCriteria'}}

In [26]:
for key in field_dict:
    print(field_dict[key].symmetric_difference(field_dict_test[key]))

set()
set()
{'transactionRevenue'}
{'campaignCode'}


The test set introduces no new columns at least.  We can go ahead and make new columns, barring advice that this is a dumb idea.